# async

In [ ]:
// // test

open testing

()



In [ ]:
open rust_operators

()



## types

In [ ]:
inl types () =
    global "#if FABLE_COMPILER // async.types"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"std::future::Future<Output = $0>\")>] type Future<'T> = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"futures::future::TryJoinAll<$0>\")>] type TryJoinAll<'T> = class end"
    global "#endif // async.types"

()



## async

In [ ]:
nominal async t = $"Async<`t>"

()



## task

In [ ]:
nominal task t =
    `(
        typecase t with
        | () => $'' : $"System.Threading.Tasks.Task"
        | _ => $'' : $"System.Threading.Tasks.Task<`t>"
    )

()



## future

In [ ]:
nominal future t = $"Future<`t>"

()



## future_pin

In [ ]:
type future_pin t = rust.pin (rust.box (rust.dyn' (future t)))

()



## future_pin_send

In [ ]:
type future_pin_send t = rust.pin (rust.box (rust.send (rust.dyn' (future t))))

()



## block_on

In [ ]:
inl block_on forall t. (fn : future_pin t) : t =
    !\($'"futures_lite::future::block_on(!fn)"')

()



## spawn

In [ ]:
inl spawn forall t. (fn : future_pin t) : t =
    !\($'"async_std::task::spawn(!fn)"')

()



## try_join_all

In [ ]:
nominal try_join_all t = $"TryJoinAll<`t>"

()



## try_join_all

In [ ]:
inl try_join_all forall t. (x : am'.vec (future_pin (resultm.result' t sm'.std_string))) : try_join_all (future_pin (resultm.result' t sm'.std_string)) =
    inl x = join x
    !\($'"futures::future::try_join_all(!x)"')

()



## await'

In [ ]:
inl await' forall t. (x : try_join_all (future_pin (resultm.result' t sm'.std_string))) : resultm.result' (am'.vec t) sm'.std_string =
    !\($'"!x.await"')

()



## await

In [ ]:
inl await forall t. (x : future_pin t) : t =
    !\($'"!x.await"')

()



## new_future

In [ ]:
inl new_future forall t. (x : () -> t) : future_pin t =
    join
        !\($'"{Box::pin(async { //"')
        x () |> fun x => join $'!x '
        !\($'"}}) //"')

()



## new_future_move

In [ ]:
inl new_future_move forall t. (x : () -> t) : future_pin t =
    join
        !\($'"{Box::pin(async move { //"')
        x () |> fun x => join $'!x '
        !\($'"}}) //"')

()



## future_init

In [ ]:
inl future_init forall t. (depth : (u8 * u8)) (flag : u8) (x : () -> t) : future_pin t =
    // join
    //     if flag = 1
    //     then new_future_move x
    //     else new_future x
    join
        if flag = 1
        then !\($'"let __result = Box::pin(async move { //"')
        else !\($'"let __result = Box::pin(async { //"')

        let x' = x ()
        inl x' = join x'

        x' |> rust.fix_closure depth

        !\($'"__result"')

()



## new_future_move_init

In [ ]:
inl new_future_move_init forall t. (depth : (u8 * u8)) (flag : u8) (x : () -> t) : future_pin t =
    future_init depth flag x
    // join
    //     !\($'"{Box::pin(async move { //"')
    //     inl x' = x () |> fun x => join $'!x '
    // 
    //         inl depth = depth |> fst
    //         if depth = 1
    //         then !\($'"!x' })"')
    //         elif depth = 2
    //         then !\($'"!x' }})"')
    //         elif depth = 3
    //         then !\($'"!x' }}})"')
    //         elif depth = 4
    //         then !\($'"!x' }}}})"')

    //         !\($'"// 1"')

()



## new_async_unit

In [ ]:
inl new_async_unit forall t. (fn : () -> ()) : async t =
    run_target function
        | Fsharp (Native) => fun () =>
            inl result : optionm'.option' (async t) = optionm'.none' ()
            $"let mutable _!result = !result"
            $"async {"
            fn ()
            $"}"
            $"|> fun x -> _!result <- Some x"
            $"_!result |> Option.get"
        | _ => fun () => null ()

()



## new_async

In [ ]:
inl new_async forall t. (fn : () -> t) : async t =
    new_async_unit (fn >> ignore)

()



## new_task

In [ ]:
inl new_task forall t. (fn : () -> t) : task t =
    run_target function
        | Fsharp (Native) => fun () =>
            inl result : optionm'.option' (task t) = optionm'.none' ()
            $"let mutable _!result = !result"
            $"task {"
            fn () |> ignore
            $"}"
            $"|> fun x -> _!result <- Some x"
            $"_!result |> Option.get"
        | _ => fun () => null ()

()



## await_task

In [ ]:
inl await_task forall t. (a : task t) : async t =
    run_target function
        | Fsharp (Native) => fun () =>
            a |> $'Async.AwaitTask'
        | _ => fun () => null ()

()



## ignore

In [ ]:
inl ignore forall t. (a : async t) : async () =
    run_target function
        | Fsharp (Native) => fun () =>
            a |> $'Async.Ignore'
        | _ => fun () => null ()

()



## run_synchronously

In [ ]:
inl run_synchronously forall t. (a : async t) : t =
    run_target function
        | Fsharp (Native) => fun () =>
            a |> $'Async.RunSynchronously'
        | _ => fun () => null ()

()



## start

In [ ]:
inl start (a : async ()) : () =
    run_target function
        | Fsharp (Native) => fun () =>
            a |> $'Async.Start'
        | _ => fun () => null ()

()



## start_child

In [ ]:
inl start_child forall t. (a : async t) : async (async t) =
    run_target function
        | Fsharp (Native) => fun () =>
            a |> $'Async.StartChild'
        | _ => fun () => null ()

()



## start_child_timeout

In [ ]:
inl start_child_timeout forall t. (timeout : i32) (a : async t) : async (async t) =
    run_target function
        | Fsharp (Native) => fun () =>
            $'Async.StartChild (!a, !timeout)'
        | _ => fun () => null ()

()



## start_immediate

In [ ]:
inl start_immediate forall t. (a : async t) : () =
    run_target function
        | Fsharp (Native) => fun () =>
            a |> $'Async.StartImmediate'
        | _ => fun () => null ()

()



## task_canceled_exception

In [ ]:
nominal task_canceled_exception = $'System.Threading.Tasks.TaskCanceledException'

()



## sleep

In [ ]:
inl sleep (ms : i32) : async () =
    run_target function
        | Fsharp (Native) => fun () =>
            ms |> $'Async.Sleep'
        | _ => fun () => null ()

()



## do

In [ ]:
inl do (a : async ()) : () =
    $'do\! !a '

()



## let'

In [ ]:
inl let' forall t. (a : async t) : t =
    $'let\! !a = !a '
    $'!a '

()



## return_await

In [ ]:
inl return_await forall t. (a : async t) : () =
    $'return\! !a '

()



## return_await'

In [ ]:
inl return_await' forall t. (a : async t) : t =
    $'return\! !a '

()



## map

In [ ]:
inl map forall t u. (fn : t -> u) (a : async t) : async u =
    fun () =>
        inl x = a |> let'
        fn x |> return
    |> new_async_unit

()



## choice

In [ ]:
union choice t1 t2 =
    | Choice1Of2 : t1
    | Choice2Of2 : t2

nominal choice' t1 t2 = $'Choice<`t1, `t2>'

()



## choice_unbox

In [ ]:
inl choice_unbox forall t1 t2. (choice : choice' t1 t2) : choice t1 t2 =
    run_target function
        | Fsharp (Native) => fun () =>
            inl choice1Of2 (x : t1) : _ _ t2 = Choice1Of2 x
            inl choice2Of2 (x : t2) : _ t1 _ = Choice2Of2 x
            $'match !choice with Choice1Of2 x -> !choice1Of2 x | Choice2Of2 x -> !choice2Of2 x'
        | _ => fun () => null ()

()



## catch'

In [ ]:
inl catch' forall t e. (a : async t) : async (choice' t e) =
    run_target function
        | Fsharp (Native) => fun () =>
            a |> $'Async.Catch'
        | _ => fun () => null ()

()



## catch

In [ ]:
inl catch forall t e. (a : async t) : async (result t e) =
    a
    |> catch'
    |> map choice_unbox
    |> map function
        | Choice1Of2 result => Ok result
        | Choice2Of2 ex => Error ex

()



## run_with_timeout_async

In [ ]:
inl run_with_timeout_async forall t. (timeout : i32) (fn : async t) : async (option t) =
    run_target function
        | Fsharp (Native) => fun () =>
            fun () =>
                inl get_locals () = $'$"timeout: {!timeout} / {!get_locals ()}"' : string
                inl child = fn |> start_child_timeout timeout |> let'
                child
                |> catch
                |> map function
                    | Ok result => Some result
                    | Error ex when ex |> sm'.format_debug |> sm'.contains "System.TimeoutException" =>
                        trace Debug (fun () => $'"run_with_timeout_async"' : string) get_locals
                        None
                    | Error (ex : exn) =>
                        inl ex = ex |> sm'.format_exception
                        trace Critical (fun () => $'$"run_with_timeout_async** / ex: {!ex}"' : string) get_locals
                        None
                |> return_await
                // $'(*'
                // $'*)'
            |> new_async_unit
        | _ => fun () => null ()

()



## run_with_timeout

In [ ]:
inl run_with_timeout timeout fn =
    fn
    |> run_with_timeout_async timeout
    |> run_synchronously

()



## cancellation_token

In [ ]:
inl cancellation_token () : async threading.cancellation_token =
    $'Async.CancellationToken'

()



In [ ]:
inl default_cancellation_token () : threading.cancellation_token =
    $'Async.DefaultCancellationToken'

()



## merge_cancellation_token_with_default_async

In [ ]:
inl merge_cancellation_token_with_default_async
    (token : threading.cancellation_token)
    : async threading.cancellation_token
    =
    run_target function
        | Fsharp (Native) => fun () =>
            fun () =>
                inl ct = cancellation_token () |> let'
                inl dct = default_cancellation_token ()
                inl cts = threading.create_linked_token_source ;[ ct; dct; token ]
                cts |> threading.cancellation_source_token |> return
            |> new_async_unit
        | _ => fun () => null ()

()



## with_trace_level

In [ ]:
inl with_trace_level forall t. level fn : _ t = new_async fun () =>
    inl trace_state = get_trace_state ()
    inl old_trace_level = *trace_state.level
    inl trace_level = trace_state.level
    try_finally
        fun () =>
            trace_level <- level
            fn |> return_await
        fun () =>
            trace_level <- old_trace_level

()



## value_task

In [ ]:
nominal value_task = $'System.Threading.Tasks.ValueTask'

()



## value_task_as_task

In [ ]:
inl value_task_as_task (task : value_task) : task () =
    $'!task.AsTask' ()

()



## await_value_task_unit

In [ ]:
inl await_value_task_unit (task : value_task) : async () =
    task |> value_task_as_task |> await_task

()



## main

In [ ]:
inl main () =
    types ()
    $"let merge_cancellation_token_with_default_async x = !merge_cancellation_token_with_default_async x" : ()

#if FABLE_COMPILER // async.types
[<Fable.Core.Erase; Fable.Core.Emit("std::future::Future<Output = $0>")>] type Future<'T> = class end
[<Fable.Core.Erase; Fable.Core.Emit("futures::future::TryJoinAll<$0>")>] type TryJoinAll<'T> = class end
#endif // async.types
let rec closure0 () (v0 : System.Threading.CancellationToken) : Async<System.Threading.CancellationToken> =
    let v1 : Async<System.Threading.CancellationToken> option = None
    let v2 : bool = true in let mutable _v1 = v1
    #if FABLE_COMPILER_RUST && !WASM && !CONTRACT
    let v3 : Async<System.Threading.CancellationToken> = null |> unbox<Async<System.Threading.CancellationToken>>
    v3
#endif
    #if FABLE_COMPILER_RUST && WASM
    let v4 : Async<System.Threading.CancellationToken> = null |> unbox<Async<System.Threading.CancellationToken>>
    v4
#endif
    #if FABLE_COMPILER_RUST && CONTRACT
    let v5 : Async<System.Threading.CancellationToken> = null |> unbox<Async<System.Threading.CancellationToken>>
    v5
#endif
 